In [2]:
import os
import json

folder_path = "../tmp/evaluation" 

# Create VisionTextDualEncoderModel

In [ ]:
from transformers import (
    VisionTextDualEncoderModel,
    VisionTextDualEncoderProcessor,
    AutoTokenizer,
    AutoImageProcessor
)
vision_model_path = "google/siglip2-base-patch16-256"
text_model_path = "../../pretrained_models/phobert_syllable/hf_phobert_syllable_base_512"
model_path = f"models/{'_'.join(vision_model_path.split('/'))}_{text_model_path.split('/')[-1].replace('hf_', '')}"
model = VisionTextDualEncoderModel.from_vision_text_pretrained(vision_model_path, text_model_path)
tokenizer = AutoTokenizer.from_pretrained(text_model_path)
image_processor = AutoImageProcessor.from_pretrained(vision_model_path)
processor = VisionTextDualEncoderProcessor(image_processor, tokenizer)
model.save_pretrained(model_path)
processor.save_pretrained(model_path)

# Copy necessary files for evaluation

In [ ]:
import os
original_model_path = "models/google_siglip-base-patch16-256-multilingual_phobert_syllable_base_512"
checkpoint_dir = "saves/google_siglip-base-patch16-256-multilingual_phobert_syllable_base_512_images_27m_CC3M_stage_2_new"
files = ["config.json", "preprocessor_config.json", "special_tokens_map.json", "tokenizer.json", "tokenizer_config.json"]
checkpoints = [checkpoint for checkpoint in os.listdir(checkpoint_dir) if "checkpoint-" in checkpoint]
checkpoints = sorted(checkpoints, key=lambda x: int(x.split('-')[1]))
checkpoints.append("")
for checkpoint in checkpoints:
    for file in files:
        os.system(f"cp {original_model_path}/{file} {checkpoint_dir}/{checkpoint}")

# Zeroshot Retrieval

## Crossmodal3600

In [11]:
data_name = "crossmodal3600" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_1"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

all_top_k = [1, 5, 10]
best_results = {}

for k in all_top_k:
    text2image_recall = f"text2image_recall@{k}"
    image2text_recall = f"image2text_recall@{k}"

    best_results[text2image_recall] = []
    best_results[image2text_recall] = []

    for checkpoint, result in all_results.items():
        best_results[text2image_recall].append({"checkpoint": checkpoint, "result": result[text2image_recall]})
        best_results[image2text_recall].append({"checkpoint": checkpoint, "result": result[image2text_recall]})

    best_results[text2image_recall] = sorted(best_results[text2image_recall], key=lambda x: x["result"], reverse=True)[:5]
    best_results[image2text_recall] = sorted(best_results[image2text_recall], key=lambda x: x["result"], reverse=True)[:5]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

162
##############################
Metric: text2image_recall@1
Best checkpoint 1: 12549
Best result 1: 0.34833332896232605
Other results:
{'text2image_recall@1': 0.34833332896232605, 'image2text_recall@1': 0.32055556774139404, 'text2image_recall@5': 0.636388897895813, 'image2text_recall@5': 0.5986111164093018, 'text2image_recall@10': 0.7477777600288391, 'image2text_recall@10': 0.7174999713897705}

Best checkpoint 2: 8633
Best result 2: 0.34805554151535034
Other results:
{'text2image_recall@1': 0.34805554151535034, 'image2text_recall@1': 0.29305556416511536, 'text2image_recall@5': 0.6230555772781372, 'image2text_recall@5': 0.5719444155693054, 'text2image_recall@10': 0.7388888597488403, 'image2text_recall@10': 0.6891666650772095}

Best checkpoint 3: 12816
Best result 3: 0.3474999964237213
Other results:
{'text2image_recall@1': 0.3474999964237213, 'image2text_recall@1': 0.3233333230018616, 'text2image_recall@5': 0.632777750492096, 'image2text_recall@5': 0.5975000262260437, 'text2image_rec

In [12]:
all_results[12994]

{'text2image_recall@1': 0.3408333361148834,
 'image2text_recall@1': 0.32111111283302307,
 'text2image_recall@5': 0.6319444179534912,
 'image2text_recall@5': 0.597777783870697,
 'text2image_recall@10': 0.7438889145851135,
 'image2text_recall@10': 0.7186111211776733}

In [13]:
all_results[13350]

{'text2image_recall@1': 0.3469444513320923,
 'image2text_recall@1': 0.3213889002799988,
 'text2image_recall@5': 0.6286110877990723,
 'image2text_recall@5': 0.5980555415153503,
 'text2image_recall@10': 0.7450000047683716,
 'image2text_recall@10': 0.7177777886390686}

In [10]:
data_name = "crossmodal3600" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_2"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

all_top_k = [1, 5, 10]
best_results = {}

for k in all_top_k:
    text2image_recall = f"text2image_recall@{k}"
    image2text_recall = f"image2text_recall@{k}"

    best_results[text2image_recall] = []
    best_results[image2text_recall] = []

    for checkpoint, result in all_results.items():
        best_results[text2image_recall].append({"checkpoint": checkpoint, "result": result[text2image_recall]})
        best_results[image2text_recall].append({"checkpoint": checkpoint, "result": result[image2text_recall]})

    best_results[text2image_recall] = sorted(best_results[text2image_recall], key=lambda x: x["result"], reverse=True)[:5]
    best_results[image2text_recall] = sorted(best_results[image2text_recall], key=lambda x: x["result"], reverse=True)[:5]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

201
##############################
Metric: text2image_recall@1
Best checkpoint 1: 12193
Best result 1: 0.6769444346427917
Other results:
{'text2image_recall@1': 0.6769444346427917, 'image2text_recall@1': 0.6780555844306946, 'text2image_recall@5': 0.9061111211776733, 'image2text_recall@5': 0.9066666960716248, 'text2image_recall@10': 0.9527778029441833, 'image2text_recall@10': 0.9574999809265137}

Best checkpoint 2: 13261
Best result 2: 0.6766666769981384
Other results:
{'text2image_recall@1': 0.6766666769981384, 'image2text_recall@1': 0.6777777671813965, 'text2image_recall@5': 0.9044444561004639, 'image2text_recall@5': 0.9055555462837219, 'text2image_recall@10': 0.9555555582046509, 'image2text_recall@10': 0.956944465637207}

Best checkpoint 3: 14062
Best result 3: 0.676111102104187
Other results:
{'text2image_recall@1': 0.676111102104187, 'image2text_recall@1': 0.6777777671813965, 'text2image_recall@5': 0.9041666388511658, 'image2text_recall@5': 0.9072222113609314, 'text2image_recall@10

In [15]:
all_results[8722]

{'text2image_recall@1': 0.6622222065925598,
 'image2text_recall@1': 0.6727777719497681,
 'text2image_recall@5': 0.9011111259460449,
 'image2text_recall@5': 0.9069444537162781,
 'text2image_recall@10': 0.9522222280502319,
 'image2text_recall@10': 0.9538888931274414}

## KTVIC

In [5]:
data_name = "KTVIC" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_1"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

all_top_k = [1, 5, 10]
best_results = {}

for k in all_top_k:
    text2image_recall = f"text2image_recall@{k}"
    image2text_recall = f"image2text_recall@{k}"

    best_results[text2image_recall] = []
    best_results[image2text_recall] = []

    for checkpoint, result in all_results.items():
        best_results[text2image_recall].append({"checkpoint": checkpoint, "result": result[text2image_recall]})
        best_results[image2text_recall].append({"checkpoint": checkpoint, "result": result[image2text_recall]})

    best_results[text2image_recall] = sorted(best_results[text2image_recall], key=lambda x: x["result"], reverse=True)[:5]
    best_results[image2text_recall] = sorted(best_results[image2text_recall], key=lambda x: x["result"], reverse=True)[:5]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

162
##############################
Metric: text2image_recall@1
Best checkpoint 1: 7832
Best result 1: 0.16487455368041992
Other results:
{'text2image_recall@1': 0.16487455368041992, 'image2text_recall@1': 0.21684588491916656, 'text2image_recall@5': 0.37849462032318115, 'image2text_recall@5': 0.41577062010765076, 'text2image_recall@10': 0.49175626039505005, 'image2text_recall@10': 0.5340501666069031}

Best checkpoint 2: 8811
Best result 2: 0.16272401809692383
Other results:
{'text2image_recall@1': 0.16272401809692383, 'image2text_recall@1': 0.2150537669658661, 'text2image_recall@5': 0.3731182813644409, 'image2text_recall@5': 0.4139784872531891, 'text2image_recall@10': 0.4910394251346588, 'image2text_recall@10': 0.5430107712745667}

Best checkpoint 3: 11748
Best result 3: 0.16272401809692383
Other results:
{'text2image_recall@1': 0.16272401809692383, 'image2text_recall@1': 0.22580644488334656, 'text2image_recall@5': 0.3698924779891968, 'image2text_recall@5': 0.421146959066391, 'text2imag

In [15]:
all_results[12994]

{'text2image_recall@1': 0.15197132527828217,
 'image2text_recall@1': 0.22580644488334656,
 'text2image_recall@5': 0.3519713282585144,
 'image2text_recall@5': 0.4229390621185303,
 'text2image_recall@10': 0.4741935431957245,
 'image2text_recall@10': 0.5250896215438843}

In [16]:
all_results[13350]

{'text2image_recall@1': 0.15161290764808655,
 'image2text_recall@1': 0.22401434183120728,
 'text2image_recall@5': 0.35555556416511536,
 'image2text_recall@5': 0.4229390621185303,
 'text2image_recall@10': 0.4767025113105774,
 'image2text_recall@10': 0.5304659605026245}

In [16]:
data_name = "KTVIC" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_2"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

all_top_k = [1, 5, 10]
best_results = {}

for k in all_top_k:
    text2image_recall = f"text2image_recall@{k}"
    image2text_recall = f"image2text_recall@{k}"

    best_results[text2image_recall] = []
    best_results[image2text_recall] = []

    for checkpoint, result in all_results.items():
        best_results[text2image_recall].append({"checkpoint": checkpoint, "result": result[text2image_recall]})
        best_results[image2text_recall].append({"checkpoint": checkpoint, "result": result[image2text_recall]})

    best_results[text2image_recall] = sorted(best_results[text2image_recall], key=lambda x: x["result"], reverse=True)[:5]
    best_results[image2text_recall] = sorted(best_results[image2text_recall], key=lambda x: x["result"], reverse=True)[:5]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

201
##############################
Metric: text2image_recall@1
Best checkpoint 1: 7743
Best result 1: 0.27706092596054077
Other results:
{'text2image_recall@1': 0.27706092596054077, 'image2text_recall@1': 0.421146959066391, 'text2image_recall@5': 0.5301074981689453, 'image2text_recall@5': 0.6469534039497375, 'text2image_recall@10': 0.6537634134292603, 'image2text_recall@10': 0.7580645084381104}

Best checkpoint 2: 9790
Best result 2: 0.27670252323150635
Other results:
{'text2image_recall@1': 0.27670252323150635, 'image2text_recall@1': 0.40860214829444885, 'text2image_recall@5': 0.5286738276481628, 'image2text_recall@5': 0.6505376100540161, 'text2image_recall@10': 0.6458781361579895, 'image2text_recall@10': 0.759856641292572}

Best checkpoint 3: 8010
Best result 3: 0.27634409070014954
Other results:
{'text2image_recall@1': 0.27634409070014954, 'image2text_recall@1': 0.41577062010765076, 'text2image_recall@5': 0.5258064270019531, 'image2text_recall@5': 0.6541218757629395, 'text2image_rec

In [18]:
all_results[8722]

{'text2image_recall@1': 0.2706093192100525,
 'image2text_recall@1': 0.41756272315979004,
 'text2image_recall@5': 0.5254480242729187,
 'image2text_recall@5': 0.6559139490127563,
 'text2image_recall@10': 0.6476702690124512,
 'image2text_recall@10': 0.774193525314331}

# Zeroshot Classification

## 30VNFoods_test

In [19]:
data_name = "30VNFoods_test" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_1"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

metrics = ["acc1", "acc5", "mean_per_class_recall"]
best_results = {}

for metric in metrics:
    best_results[metric] = []

    for checkpoint, result in all_results.items():
        best_results[metric].append({"checkpoint": checkpoint, "result": result[metric]})

    best_results[metric] = sorted(best_results[metric], key=lambda x: x["result"], reverse=True)[0: 10]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

162
##############################
Metric: acc1
Best checkpoint 1: final
Best result 1: 0.12301587301587301
Other results:
{'acc1': 0.12301587301587301, 'acc5': 0.39920634920634923, 'mean_per_class_recall': 0.09757170432492085}

Best checkpoint 2: 13706
Best result 2: 0.12202380952380952
Other results:
{'acc1': 0.12202380952380952, 'acc5': 0.4007936507936508, 'mean_per_class_recall': 0.09751268448736641}

Best checkpoint 3: 14151
Best result 3: 0.12202380952380952
Other results:
{'acc1': 0.12202380952380952, 'acc5': 0.39900793650793653, 'mean_per_class_recall': 0.09749507572578223}

Best checkpoint 4: 14329
Best result 4: 0.12202380952380952
Other results:
{'acc1': 0.12202380952380952, 'acc5': 0.39900793650793653, 'mean_per_class_recall': 0.0974545057985255}

Best checkpoint 5: 12816
Best result 5: 0.12182539682539682
Other results:
{'acc1': 0.12182539682539682, 'acc5': 0.3996031746031746, 'mean_per_class_recall': 0.09732651132056587}

Best checkpoint 6: 12905
Best result 6: 0.12142857

In [22]:
all_results[8722]

{'acc1': 0.09662698412698413,
 'acc5': 0.39325396825396824,
 'mean_per_class_recall': 0.08241780395101662}

In [23]:
data_name = "30VNFoods_test" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_2"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

metrics = ["acc1", "acc5", "mean_per_class_recall"]
best_results = {}

for metric in metrics:
    best_results[metric] = []

    for checkpoint, result in all_results.items():
        best_results[metric].append({"checkpoint": checkpoint, "result": result[metric]})

    best_results[metric] = sorted(best_results[metric], key=lambda x: x["result"], reverse=True)[0: 10]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

138
##############################
Metric: acc1
Best checkpoint 1: 3471
Best result 1: 0.19484126984126984
Other results:
{'acc1': 0.19484126984126984, 'acc5': 0.5136904761904761, 'mean_per_class_recall': 0.18095694485476574}

Best checkpoint 2: 3738
Best result 2: 0.19206349206349208
Other results:
{'acc1': 0.19206349206349208, 'acc5': 0.5128968253968254, 'mean_per_class_recall': 0.17819058593314782}

Best checkpoint 3: 3382
Best result 3: 0.19126984126984126
Other results:
{'acc1': 0.19126984126984126, 'acc5': 0.4982142857142857, 'mean_per_class_recall': 0.1761696435046797}

Best checkpoint 4: 1869
Best result 4: 0.1873015873015873
Other results:
{'acc1': 0.1873015873015873, 'acc5': 0.501984126984127, 'mean_per_class_recall': 0.18189465990155088}

Best checkpoint 5: 3560
Best result 5: 0.1869047619047619
Other results:
{'acc1': 0.1869047619047619, 'acc5': 0.5111111111111111, 'mean_per_class_recall': 0.1725536626349396}

Best checkpoint 6: 3827
Best result 6: 0.1863095238095238
Other 

In [24]:
all_results[8722]

{'acc1': 0.17083333333333334,
 'acc5': 0.49662698412698414,
 'mean_per_class_recall': 0.15350473255696329}

## 30VNFoods_val

In [20]:
data_name = "30VNFoods_val" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_1"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

metrics = ["acc1", "acc5", "mean_per_class_recall"]
best_results = {}

for metric in metrics:
    best_results[metric] = []

    for checkpoint, result in all_results.items():
        best_results[metric].append({"checkpoint": checkpoint, "result": result[metric]})

    best_results[metric] = sorted(best_results[metric], key=lambda x: x["result"], reverse=True)[0: 10]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

162
##############################
Metric: acc1
Best checkpoint 1: 14151
Best result 1: 0.11292246520874752
Other results:
{'acc1': 0.11292246520874752, 'acc5': 0.3912524850894632, 'mean_per_class_recall': 0.09218440927734792}

Best checkpoint 2: final
Best result 2: 0.11252485089463221
Other results:
{'acc1': 0.11252485089463221, 'acc5': 0.3908548707753479, 'mean_per_class_recall': 0.09175108297092767}

Best checkpoint 3: 12994
Best result 3: 0.1117296222664016
Other results:
{'acc1': 0.1117296222664016, 'acc5': 0.39363817097415504, 'mean_per_class_recall': 0.09139738024813086}

Best checkpoint 4: 13083
Best result 4: 0.1117296222664016
Other results:
{'acc1': 0.1117296222664016, 'acc5': 0.39284294234592443, 'mean_per_class_recall': 0.09165423433683201}

Best checkpoint 5: 12371
Best result 5: 0.11133200795228629
Other results:
{'acc1': 0.11133200795228629, 'acc5': 0.39324055666003976, 'mean_per_class_recall': 0.09181378418236097}

Best checkpoint 6: 12727
Best result 6: 0.11133200795

In [21]:
all_results[13350]

{'acc1': 0.10815109343936381,
 'acc5': 0.39244532803180915,
 'mean_per_class_recall': 0.088496748633413}

In [25]:
data_name = "30VNFoods_val" 
model_name = "google_siglip2-base-patch16-256"
stage_name = "stage_2"

files = sorted([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" in file], key=lambda file: int(file.split("checkpoint-")[1].split("_hf")[0]))
files.extend([file for file in os.listdir(folder_path) if data_name in file and model_name in file and "phobert" in file and stage_name in file and "checkpoint" not in file])
print(len(files))

all_results = {}
for file in files:
    if "checkpoint" in file:
        checkpoint = int(file.split("checkpoint-")[1].split("_hf")[0])
    else:
        checkpoint = "final"

    with open(f"{folder_path}/{file}") as f:
        result = json.load(f)

    all_results[checkpoint] = result

metrics = ["acc1", "acc5", "mean_per_class_recall"]
best_results = {}

for metric in metrics:
    best_results[metric] = []

    for checkpoint, result in all_results.items():
        best_results[metric].append({"checkpoint": checkpoint, "result": result[metric]})

    best_results[metric] = sorted(best_results[metric], key=lambda x: x["result"], reverse=True)[0: 10]

for metric, top_results in best_results.items():
    print("##########"*3)
    print(f"Metric: {metric}")
    for i, best_result in enumerate(top_results):
        print(f"Best checkpoint {i+1}: {best_result['checkpoint']}")
        print(f"Best result {i+1}: {best_result['result']}")
        print(f"Other results:")
        print(all_results[best_result['checkpoint']]) 
        print()

201
##############################
Metric: acc1
Best checkpoint 1: 3738
Best result 1: 0.18807157057654075
Other results:
{'acc1': 0.18807157057654075, 'acc5': 0.5009940357852882, 'mean_per_class_recall': 0.17336460887054805}

Best checkpoint 2: 1869
Best result 2: 0.18608349900596421
Other results:
{'acc1': 0.18608349900596421, 'acc5': 0.4970178926441352, 'mean_per_class_recall': 0.17963105071314764}

Best checkpoint 3: 3471
Best result 3: 0.18409542743538768
Other results:
{'acc1': 0.18409542743538768, 'acc5': 0.49940357852882705, 'mean_per_class_recall': 0.17043808771275962}

Best checkpoint 4: 4094
Best result 4: 0.18409542743538768
Other results:
{'acc1': 0.18409542743538768, 'acc5': 0.4834990059642147, 'mean_per_class_recall': 0.16671315954804794}

Best checkpoint 5: 8099
Best result 5: 0.18369781312127237
Other results:
{'acc1': 0.18369781312127237, 'acc5': 0.48667992047713715, 'mean_per_class_recall': 0.1653871945294764}

Best checkpoint 6: 3382
Best result 6: 0.183300198807157

In [26]:
all_results[8722]

{'acc1': 0.17296222664015903,
 'acc5': 0.495427435387674,
 'mean_per_class_recall': 0.1529775856085647}